# Introdução ao MapReduce


## Na Prática

### `mapper.py`

In [5]:
#!/usr/bin/env python

import sys

# Lê cada linha da entrada padrão
for line in sys.stdin:
    
    # Obtem as palavras em cada linha
    words = line.split(' ')
    
    # Gera o contador para cada palavra
    for word in words:
        
        # Escreve o par chave-valor para stdout (saida padrão) para a ser processado pelo reducer
        # A chave é qualquer coisa antes do primeiro caracter de tab o e valor 
        # (#valor) é qualquer coisa após o caracter de tab
        print('{0}\t{1}'.format(word, 1))


### `reducer.py`

In [4]:
#!/usr/bin/env python

import sys

curr_word = None
curr_count = 0

# Processa cada chave-valor do mapper

for line in sys.stdin:
    # Obtem a chave e valor da linha atual
    word, count = line.split('\t')
    
    # Converte a contagem em um inteiro
    count = int(count)
    
    # Se a palavra atual é a mesma que a palavra anterior,
    # incremente a contagem. Do contrário, imprima as palavras em stdout (saída  padrão)
    if (word == curr_word):
        curr_count += count
    else:
    # Escreva a palavra e o número de ocorrências como um chave-valor em stdout
        if curr_word:
            print('{0}\t{1}'.format(curr_word, curr_count))
            curr_count = count
    
    # Imprime a contagem para a última palavra
    print('{0}\t{1}'.format(curr_word, curr_count))
    

Antes de executar o código no terminal, verifique as permissões nos arquivos `mapper.py` e `reducer.py`

`chmod a+x mapper.py reducer.py`

### Teste

* Teste os programas antes de rodá-los como tarefas MapReduce

* Podemos utilizar comandos `echo` concatenados por `pipes`

`echo 'jack be nimble jack be quick' | ./mapper.py | sort -t 1 | reducer.py`

### mrjob

* É uma biblioteca Python MapReduce criada pela Yelp


* Permite a criação das tarefas utilizando as facilidades Python


* Podem ser testadas localmente, no Hadoop Cluster ou ainda na nuvem utilizando a Amazon Elastic MapReduce (EMR)


* Desenvolvido ativamente com múltiplos commits toda semana


* Possui documentação abrangente, mais que todos os outros frameworks ou bibliotecas que suportem o MapReduce


* Permite que as aplicações possam ser executadas e testadas sem que o Hadoop tenha que ser instalado, o que ajuda no desenvolvimento e testes antes do deploy no Cluster Hadoop


* Não dá o mesmo nível de acesso ao Hadoop que as outras APIs oferecem

#### Instalação

`pip install mrjob`

#### Contagem de Palavras no `mrjob`

1. Abra um terminal e vá até o diretório em que pretende executar a tarefa MapReduce


2. Digite os comandos abaixo

`touch input.txt`

`echo "jack be nimble jack be quick" >> input.txt`

3. Escreva uma classe chamada `word_count.py` Python como a seguir

In [8]:
from mrjob.job import MRJob

class MRWordCount(MRJob):
    
    def mapper(self, _, line):
        for word in line.split():
            yield(word, 1)
            
    def reducer(self, word, counts):
        yield(word, sum(counts))

if (__name__ == '__name__'):
    MRWordCount.run()

4. Execução

`python word_count.py input.txt`

#### O que está acontecendo?

* Herança de MRJob


* `mapper()`

  => O `_` ignora a entrada e divide o valor de entrada para produzir as palavras e as contagens
  
  
* `combiner()` 

  => Representa um combinador para a tarefa MapReduce
  
  => É um processo que roda após o mapper e antes do reducer
  
  
* `reducer()`

#### Executando tarefas mrjob

* Executar tarefas MapReduce com mrjob é similar a executar scripts python

  `python mr_job.py input.txt`
  
  
* Múltiplos arquivos podem ser passados como argumentos, bastando especificar seus nomes na linha de comando:

  `python mr_job.py input1.txt input2.txt. input3.txt`
  
  
* mrjob também aceita entradas pela entrada padrão:

  `python mr_job.py < input.txt`

### Os Salários Mais Altos

In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
import csv

cols = 'Name,JobTitle,AgencyID,Agency,HireDate,AnnualSalary,Gross Pay'.split(',')

class salarymax(MRJob):
    
    def mapper(self, _, line):
        #Converte cada linha em um dicionário
        row = dict(zip(cols, [ a.strip() for a in csv.reader([line]).next()]))
        
        # Faz o Yield do salário
        yield 'salary', (float(row['AnnualSalary'][1:]), line)
        
        try:
            yield 'gross', (float(row['GrossPay'][1:]), line) 
        except ValueError:
            self.increment_counter('warn', 'missing gross', 1)
            
            
    def reducer(self, key, values):
        topten = []
        
        for p in values:
            topten.append(p) 
            topten.sort() 
            topten = topten[-10:]

            for p in topten:
                yield key, p
                combiner = reducer

if (__name__ == '__main__'):
    salarymax.run()